In [34]:
from pymongo import MongoClient, InsertOne
from prettytable import PrettyTable

client = MongoClient('mongodb://localhost:27017/')
db = client['university_homework'] # Database university_homework
courses = db['courses']  # Courses table
students = db['students'] # Students table

# Courses Data
courses_operations = [
    InsertOne({'course': 'Data Structures', 'enrollments': 50, 'department': 'Computer Science'}),
    InsertOne({'course': 'Machine Learning', 'enrollments': 40, 'department': 'Computer Science'}),
    InsertOne({'course': 'Calculus I', 'enrollments': 55, 'department': 'Mathematics'}),
    InsertOne({'course': 'Linear Algebra', 'enrollments': 45, 'department': 'Mathematics'}),
    InsertOne({'course': 'Physics 101', 'enrollments': 30, 'department': 'Physics'}),
    InsertOne({'course': 'Quantum Mechanics', 'enrollments': 20, 'department': 'Physics'}),
    InsertOne({'course': 'Thermodynamics', 'enrollments': 25, 'department': 'Engineering Physics'}),
]
# Insert courses data to courses table
courses.bulk_write(courses_operations)

# Students data
students_operations = [
    InsertOne({'name': 'Alice', 'course': 'Data Structures'}),
    InsertOne({'name': 'Bob', 'course': 'Data Structures'}),
    InsertOne({'name': 'Charlie', 'course': 'Machine Learning'}),
    InsertOne({'name': 'David', 'course': 'Calculus I'}),
    InsertOne({'name': 'Eve', 'course': 'Linear Algebra'}),
    InsertOne({'name': 'Frank', 'course': 'Physics 101'}),
    InsertOne({'name': 'Grace', 'course': 'Quantum Mechanics'}),
    InsertOne({'name': 'Heidi', 'course': 'Thermodynamics'}),
]
# Insert students data to students table
students.bulk_write(students_operations)

# Show courses table
courses_data = courses.find()
table = PrettyTable()
table.field_names = ["Course", "Department", "Enrollments"]

for c in courses_data:
    table.add_row([c['course'], c['department'], c['enrollments']])

print("Courses Table:")
print(table)

# Show students table
students_data = students.find()
table2 = PrettyTable()
table2.field_names = ["Student Name", "Course"]

for s in students_data:
    table2.add_row([s['name'], s['course']])

print("\nStudents Table:")
print(table2)

Courses Table:
+-------------------+---------------------+-------------+
|       Course      |      Department     | Enrollments |
+-------------------+---------------------+-------------+
|  Data Structures  |   Computer Science  |      50     |
|  Machine Learning |   Computer Science  |      40     |
|     Calculus I    |     Mathematics     |      55     |
|   Linear Algebra  |     Mathematics     |      45     |
|    Physics 101    |       Physics       |      30     |
| Quantum Mechanics |       Physics       |      20     |
|   Thermodynamics  | Engineering Physics |      25     |
+-------------------+---------------------+-------------+

Students Table:
+--------------+-------------------+
| Student Name |       Course      |
+--------------+-------------------+
|    Alice     |  Data Structures  |
|     Bob      |  Data Structures  |
|   Charlie    |  Machine Learning |
|    David     |     Calculus I    |
|     Eve      |   Linear Algebra  |
|    Frank     |    Physics 101   

In [38]:
# Query count courses per department
pipeline = [
    {'$group': {'_id': '$department', 'count_courses': {'$sum': 1}}}
]
# Collect data from query above
count_courses = courses.aggregate(pipeline)

# Show data with table
table = PrettyTable()
table.field_names = ["Department", "Total Course"]
for i in count_courses:
    table.add_row([i['_id'], i['count_courses']])
print(table)

+---------------------+--------------+
|      Department     | Total Course |
+---------------------+--------------+
|       Physics       |      2       |
|     Mathematics     |      2       |
| Engineering Physics |      1       |
|   Computer Science  |      2       |
+---------------------+--------------+


In [37]:
# Query for get courses with enrollments over 25 in Computer Science
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$group': {'_id': '$department','total_courses': {'$sum': 1},'total_enrollments': {'$sum': '$enrollments'}}}
]
# Collect data from query above
result = courses.aggregate(pipeline)

# Show data with table
table = PrettyTable()
table.field_names = ["Department", "Total Courses", "Total Enrollments"]

for i in result:
    table.add_row([i['_id'], i['total_courses'], i['total_enrollments']])
    
print("Computer Science Courses with Enrollments > 25:")
print(table)

Computer Science Courses with Enrollments > 25:
+------------------+---------------+-------------------+
|    Department    | Total Courses | Total Enrollments |
+------------------+---------------+-------------------+
| Computer Science |       2       |         90        |
+------------------+---------------+-------------------+


In [36]:
# Query for get courses with enrollments greater than or equal to 25 in Computer Science
pipeline_gte_group = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gte': 25}}},
    {'$group': {'_id': '$department','total_courses': {'$sum': 1},'total_enrollments': {'$sum': '$enrollments'}}}
]
# Collect data from query above
result_gte_group = courses.aggregate(pipeline_gte_group)

# Show data with table
table = PrettyTable()
table.field_names = ["Department", "Total Courses", "Total Enrollments"]

for r in result_gte_group:
    table.add_row([r['_id'], r['total_courses'], r['total_enrollments']])

print("Computer Science Courses with Enrollments >= 25:")
print(table)

Computer Science Courses with Enrollments >= 25:
+------------------+---------------+-------------------+
|    Department    | Total Courses | Total Enrollments |
+------------------+---------------+-------------------+
| Computer Science |       2       |         90        |
+------------------+---------------+-------------------+


In [35]:
# Query lookup for join Courses with Students
pipeline = [
    {'$lookup': {'from': 'students','localField': 'course','foreignField': 'course','as': 'students_list'}}
]

courses_with_students = courses.aggregate(pipeline)

# Tampilkan dengan PrettyTable
table = PrettyTable()
table.field_names = ["Course", "Department", "Enrollments", "Students"]

for c in courses_with_students:
    # ambil nama student saja
    student_names = [s['name'] for s in c['students_list']]
    table.add_row([c['course'], c['department'], c['enrollments'], ", ".join(student_names)])

print("Courses with Students:")
print(table)


Courses with Students:
+-------------------+---------------------+-------------+------------+
|       Course      |      Department     | Enrollments |  Students  |
+-------------------+---------------------+-------------+------------+
|  Data Structures  |   Computer Science  |      50     | Alice, Bob |
|  Machine Learning |   Computer Science  |      40     |  Charlie   |
|     Calculus I    |     Mathematics     |      55     |   David    |
|   Linear Algebra  |     Mathematics     |      45     |    Eve     |
|    Physics 101    |       Physics       |      30     |   Frank    |
| Quantum Mechanics |       Physics       |      20     |   Grace    |
|   Thermodynamics  | Engineering Physics |      25     |   Heidi    |
+-------------------+---------------------+-------------+------------+


In [ ]:
courses_col.delete_many({})
students_col.delete_many({})